<a href="https://colab.research.google.com/github/thxsxth/POMDP_RLSepsis/blob/master/RL_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
cd 'drive/My Drive/sepsis3-cohort'

/content/drive/My Drive/sepsis3-cohort


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
import torch
import numpy as np
import pandas as pd
import datetime as dt
import random
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import os
import glob
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence,pad_packed_sequence
from autoencoder_models import Encoder,patient_encoder,Transition,mechanistic
device='cuda' if torch.cuda.is_available() else 'cpu'
from load_and_process import *

print('Loading Done')

Loading Done


In [42]:
labs=labs[['ICUSTAY_ID','charttime','ANIONGAP','BICARBONATE',	'CREATININE',	'CHLORIDE',	'GLUCOSE',	'HEMATOCRIT',	
           'HEMOGLOBIN','PLATELET'	,'POTASSIUM',	'SODIUM',	'BUN'	,'WBC']]

cols=['ANIONGAP','BICARBONATE',	'CREATININE',	'CHLORIDE',	'GLUCOSE',	'HEMATOCRIT',	
           'HEMOGLOBIN','PLATELET'	,'POTASSIUM',	'SODIUM',	'BUN'	,'WBC']
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
labs[cols]=scaler.fit_transform(labs[cols])

print('Labs Rescaled')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


Labs Rescaled


Get the Vaso Pressor csv files, MV is divided into patient hourly bins, but CV gives only charttime

In [4]:
vaso_cv=pd.read_csv('./pivoted_sofa/Vaso_CV.csv',parse_dates=['charttime'])
vaso_MV=pd.read_csv('./pivoted_sofa/Vaso_MV.csv',parse_dates=['endtime','starttime'])

vaso_cv['starttime']=vaso_cv['charttime']


In [39]:
vaso_cv['rate_std']=vaso_cv['rate_std'].apply(lambda x: 1e-4 if x==0 else x)

In [40]:
vaso_cv['rate_std'].min()

0.0001

Patient List (Selected w.r.t NA % and Sepsis3+Death)

In [5]:

RL_pat_list=list(np.load('RL_Pat_list.npy'))

## Goal is to get a Replay Buffer

Need the states, hourly includes Vitals, Sofa Scores, Labs and the hidden CV hidden Labs

### Redefining some modules in a more convenient manner  

In [6]:

class Encoder_labs(nn.Module):
  """ Encodes x_{:t} 
      The job of the Encoder is to remember the past observations.
     
  """

  def __init__(self,input_dim,h_dim,n_layers=12, dropout=0.0):
      
      super(Encoder_labs,self).__init__()
      
      self.rnn=nn.GRU(input_dim,h_dim,n_layers,batch_first=True)
      self.dropout=dropout  
      self.n_layers=n_layers
      self.hidden_dim=h_dim
      self.init_weights()
      

  def  init_weights(self):
        for w in self.rnn.parameters(): # initialize the gate weights with orthogonal
            if w.dim()>1:
                weight_init.orthogonal_(w)

  def forward(self,obs,obs_lens,init_h=None, noise=False):

    
    batch_size, max_len, freq=obs.size()  
    obs_lens=torch.LongTensor(obs_lens).to(device)
    obs_lens_sorted, indices = obs_lens.sort(descending=True)
    obs_sorted = obs.index_select(0, indices).to(device)
    
    packed_obs=pack_padded_sequence(obs_sorted,obs_lens_sorted.data.tolist(),batch_first=True)
    # if init_h is None:
    #     init_h=self.init_h

     
    
    hids, h_n = self.rnn(packed_obs) # hids: [B x T x H]  
                                                  # h_n: [num_layers*B*H)
    _, inv_indices = indices.sort()

    hids, lens = pad_packed_sequence(hids, batch_first=True)         
    hids = hids.index_select(0, inv_indices) #B*T*H
    
    
            
    return hids

In [7]:
class stacked_autoencoder_new(nn.Module):
  """
  Denoising autoencoder adjusted so it will return the hidden state

  """

  def __init__(self,input_dim,h_dims=[512,128,10],n_layers=[12,8,3]):
    
    super(stacked_autoencoder_new,self).__init__()   
    self.encoder1=Encoder_labs(input_dim,h_dims[0],n_layers[0])
    self.encoder2=Encoder_labs(h_dims[0],h_dims[1],n_layers[1])
    self.encoder3=Encoder_labs(h_dims[1],h_dims[2],n_layers[2])
    self.linear=nn.Linear(h_dims[2],input_dim)


  def forward(self,obs,obs_lens):

    out1=self.encoder1(obs,obs_lens)  #B*T*h_1
    out1=pad_sequence(out1,batch_first=True)

    out2=self.encoder2(out1,obs_lens)  #B*T*h_1
    out2=pad_sequence(out2,batch_first=True) #B*T*h_2

    out3=self.encoder3(out2,obs_lens)
    out3=pad_sequence(out3,batch_first=True) #B*T*h_3
    
    output_=self.linear(out3)

    return output_,out3

In [8]:
stacked_rnn=stacked_autoencoder_new(12,h_dims=[128,64,10],n_layers=[8,5,3])
if device=='cuda':
  stacked_rnn.cuda()

stacked_rnn.load_state_dict(torch.load('./lab_rnn/stacked__lab_rnn0.5.pt',map_location='cpu'))

<All keys matched successfully>

In [11]:
class state_space_new(nn.Module):
  
  def __init__(self,z_dim,obs_dim,full_dim,h_dim,a_dim,constants_dim):
    
    super(state_space_new,self).__init__()

    self.rnn=Encoder(full_dim,h_dim,a_dim)
    self.trans=Transition(z_dim,a_dim,h_dim,constants_dim)
    self.z_baselines=torch.FloatTensor([70.0,1.0,2.0,0.7,70.0]).to(device)
    self.initial=patient_encoder(constants_dim,z_dim,h_dim)
    
    if device=='cuda':
      self.cuda()

  def forward(self,trajectories,treatments,constants,seq_lens,obs):
      """
      trajectories : Batch of Full_observations B*T*|Full_O|
      seq_lengths :list
      actions : B*T*|A|
      Constants B*|C|
      obs : B*T*|O|
      others : B*T*(|Full_O|-|O|)
      """
      
      T_max =trajectories.size(1)
      batch_size=trajectories.size(0)
      baselines=self.z_baselines.expand(batch_size,-1)
      z_prev,h_prev=self.initial(constants)
    
      h_prev=h_prev.contiguous()
      
      outputs=[]
      cv_params=[]

      a_prev=torch.zeros(treatments.shape[0],treatments.shape[2]).to(device) #B*A
      rnn_output=self.rnn(trajectories,seq_lens,constants,h_prev) 
   
       #rnn_ouput has shape B*T*H

      # we enclose all the sample statements in the model in a plate.
      # this marks that each datapoint is conditionally independent of the others
      
      for t in range(1,T_max+1):

        z_t=self.trans(z_prev,a_prev,rnn_output[:,t-1,:],constants)
        z_scaled=z_t+baselines
        
        o_t=mechanistic(z_scaled) #B*obs_dim
        outputs.append(o_t)
        cv_params.append(z_scaled)
        
        z_prev = z_t
        a_prev=treatments[:,t-1,:]
         
           
      temp=torch.stack(outputs,dim=1).to(device)
      cv=torch.stack(cv_params,dim=1).to(device)
      return temp,cv


model=state_space_new(z_dim=5,obs_dim=4,full_dim=12,h_dim=64,a_dim=2,constants_dim=3)
# model.load_state_dict(torch.load('./physio/0.1_denoise_auto_17.763059199264546.pt'))
model.load_state_dict(torch.load('./physio/0.1_denoise_auto_17.763059199264546.pt'))

<All keys matched successfully>

In [10]:
ls physio

0.1_denoise_auto_17.763059199264546.pt  denoise_auto_20.698103513470638.pt
0.1_denoise_auto_18.199223333661593.pt  denoise_auto_21.14946742157139.pt
denoise_auto_0.15.pt                    denoise_auto_21.27327754376229.pt
denoise_auto_0.5.pt                     denoise_auto_22.28249618758459.pt
denoise_auto_19.03568517701071.pt       denoise_auto_24.59860025552893.pt
denoise_auto_19.292082309881536.pt      denoise_auto_24.798284542387346.pt
denoise_auto_19.47176710134285.pt       denoise_auto_24.989851851415093.pt
denoise_auto_20.017858770721816.pt      denoise_auto_25.261256060555127.pt
denoise_auto_20.442816033995523.pt      denoise_auto_27.926003456642675.pt


## Helper Functions to get Rewards and the Full Trajectory of a patient

In [12]:
C1,C2=-0.125,-0.025
def get_rewards(df,dead):
  """
  Get's rewards for a trajectory
  df_sofa: Pandas Df which contains SOFA scores
  MUST BE REINDEXED to have time
  dead (bool): If the terminal is survival or death
  """
  # Calculate rewards for SOFA_{t+1} -SOFA_{t}

  # rewards1=C1*(df.SOFA_24hours-df.shift().SOFA_24hours).dropna().values
  rewards1=C1*(df.SOFA_24hours.values[1:]-df.SOFA_24hours.values[:-1])
  # print(rewards1.shape)
  
  rewards2=C2*((df.shift().SOFA_24hours.iloc[1:]==df.SOFA_24hours.iloc[1:])&(df.SOFA_24hours.iloc[1:]>0)).astype('int').values
  # print(rewards2.shape)
  
  rewards=rewards1+rewards2
  # Calculate Terminal rewards
  if dead:
    rewards=np.concatenate([rewards,[-15]])
  else:
    rewards=np.concatenate([rewards,[15]])

  return rewards


4+4

8

We have a patient, so we need to get the 'hidden states', ( all timed in order) and rewards and buffer should contain (s,a,s',r,done)

For each patient we can get hidden lab states +hidden cv_states

In [ ]:
pat=RL_pat_list[299]

In [13]:
def get_lab_hs(pat_labs):
   """

   Takes patient specific lab dataframe and returns labs with hidden values concat'd
   Input : pd.Dataframe, index: Charttime, and cols [['charttime','ANIONGAP','BICARBONATE',	'CREATININE',	'CHLORIDE',	'GLUCOSE',	'HEMATOCRIT',	
           'HEMOGLOBIN','PLATELET'	,'POTASSIUM',	'SODIUM',	'BUN'	,'WBC']]
   Output : pd.Dataframe with index:Charttime

   """
   lab_vals=torch.Tensor(pat_labs.values).to(device).unsqueeze(0)
   seq_len=[lab_vals.shape[1]]

   with torch.no_grad():
     _,h=stacked_rnn(lab_vals,seq_len)

   labs_h=pd.DataFrame(np.concatenate([pat_labs.values,h.squeeze(0).to('cpu').numpy()],axis=1),index=pat_labs.index) 

   return labs_h
 
# pat_lab=labs[labs.ICUSTAY_ID==pat]
# pat_lab=pat_lab[['charttime','ANIONGAP','BICARBONATE',	'CREATININE',	'CHLORIDE',	'GLUCOSE',	'HEMATOCRIT',	
#            'HEMOGLOBIN','PLATELET'	,'POTASSIUM',	'SODIUM',	'BUN'	,'WBC']]

# pat_lab=pat_lab.set_index('charttime').resample('12H').mean().ffill().dropna()


# pat_labs=get_lab_hs(pat_lab)
# pat_labs.head()

In [14]:
def get_trajectory(pat):
  #get fluids This will give Volume, for the patient, indexed by charttiime
  pat_fluids=input_fluids[input_fluids.icustay_id==pat].set_index('charttime')
  
  #make sure sofa is hourly 
  pat_sofa=sofa[sofa.icustay_id==pat].set_index('endtime').resample('H').last().ffill()

  
  #Vasopressors also come from CV and MV data sources both
  """
  As mentioned earlier, CV has charttime whilst MV is already hourly
  """
  
  pat_vaso=pd.concat([vaso_cv[vaso_cv.icustay_id==pat].set_index('starttime').resample('H').max(),
           vaso_MV[vaso_MV.icustay_id==pat].set_index('starttime').resample('H').mean().fillna(0)]).resample('H').last().fillna(0)

  
  pat_vaso['Vaso_rate']=pat_vaso['rate_std']+pat_vaso['vaso_rate']

  pat_vaso=pat_vaso[['icustay_id','Vaso_rate']]
  
  
  #Here we use .sum() so that all fluids within a hr will be added, pat_vaso and pat_sofa are already resampled so won't be an issue

  pat_sofa=pd.concat([pat_sofa,pat_fluids,pat_vaso]).resample('H').sum()[['liver_24hours','cardiovascular_24hours',
                                      'cns_24hours','renal_24hours','SOFA_24hours','Vaso_rate','volume']]

  pat_vitals=vitals[vitals.icustay_id==pat].set_index('charttime')[['HeartRate','SysBP','DiasBP',	'MeanBP','RespRate','SpO2','TempC']]
  

  pat_lab=labs[labs.ICUSTAY_ID==pat]
  pat_lab=pat_lab[['charttime','ANIONGAP','BICARBONATE',	'CREATININE',	'CHLORIDE',	'GLUCOSE',	'HEMATOCRIT',	
           'HEMOGLOBIN','PLATELET'	,'POTASSIUM',	'SODIUM',	'BUN'	,'WBC']]

  ## To get the hidden states of denoising autoencoder we need labs to be 12H 
  pat_lab=pat_lab.set_index('charttime').resample('12H').mean().ffill().dropna()

  #If no labs don't consider this patient
  if pat_lab.shape[0]<1:
    return None
  
  
  
  labs_with_h=get_lab_hs(pat_lab)



  pat_df=pd.concat([pat_vitals,
                              pat_sofa,labs_with_h]).resample('H').last().ffill().dropna()

 
  dead=co.loc[pat].HOSPITAL_EXPIRE_FLAG==1
  pat_df=pat_df.truncate(before=co.loc[pat].intime-pd.Timedelta('1hr'))

  if co.loc[pat].HOSPITAL_EXPIRE_FLAG==1:
       pat_df=pat_df.truncate(after=co.loc[pat].death_time)
  else:
    pat_df=pat_df.truncate(after=co.loc[pat].outtime)
  
  if pat_df.shape[0]<2:
    return None

  ### This part is to get CV states using the Cardiovascular Autoencoder
  
  treatments=torch.FloatTensor(pat_df[['Vaso_rate','volume']].values).to(device).unsqueeze(0)     

  trajectory=torch.FloatTensor(pat_df[['HeartRate','SysBP','DiasBP',	'MeanBP','RespRate','SpO2','TempC',
                                      'liver_24hours','cardiovascular_24hours',
                                      'cns_24hours','renal_24hours','SOFA_24hours']].values).to(device).unsqueeze(0)


  obs=torch.FloatTensor(pat_df[['HeartRate','SysBP','DiasBP',	'MeanBP']].values).to(device).unsqueeze(0)
  constants=torch.FloatTensor(co.loc[pat][['age','is_male','Weight']]).to(device).unsqueeze(0)
  
  seq_len=[pat_df.shape[0]]

  with torch.no_grad():
    _,cvs=model(trajectory,treatments,constants,seq_len,obs)

  full_states=pd.DataFrame(np.concatenate([pat_df.values,cvs[:,:,1:].squeeze(0).to('cpu').numpy()],axis=1),
                         index=pat_df.index,columns=list(pat_df.columns)+['R','C','T','SV'])

  full_states['age']=co.loc[pat].age
  full_states['gender']=co.loc[pat].is_male
  full_states['weight']=co.loc[pat].Weight

  treatments=full_states[['Vaso_rate','volume']].values
  rewards=get_rewards(pat_df,dead)
  full_states=full_states.drop(['Vaso_rate','volume'],axis=1)
  # print(full_states)
  
  return full_states,treatments,rewards


print('Done')

Done


## Loop through and save as a csv

In [43]:
buffer=[]
max_sofa=0
pat=0
"""
buffer should contain (s,s',a,r,done)
is done=T, s'=None
"""

for i,pat in enumerate(RL_pat_list):
   output=get_trajectory(pat)
   if output:
     full_states,treatments,rewards=output
     T=full_states.shape[0]

     full_states=full_states.values
     if T<2:
       continue
     
     
     for t in range(1,T+1):
        
        a=treatments[t-1,:]
        s=full_states[t-1,:]
        
        r=rewards[t-1]
        if t==T:
            # s1=None
            done=True

        else:
          #  s1=full_states[t,:]
           done=False
        
        # Appends a 5,tuple
        buffer.append((pat,s,a,r,done))

    
   
   print(i/len(RL_pat_list)*100,'% Done')


Streaming output truncated to the last 5000 lines.
73.10485823424975 % Done
73.11023833862377 % Done
73.1156184429978 % Done
73.12099854737181 % Done
73.12637865174585 % Done
73.13175875611986 % Done
73.13713886049389 % Done
73.14251896486792 % Done
73.14789906924194 % Done
73.15327917361597 % Done
73.15865927798998 % Done
73.16403938236402 % Done
73.16941948673804 % Done
73.17479959111208 % Done
73.18017969548609 % Done
73.18555979986013 % Done
73.19093990423414 % Done
73.19632000860817 % Done
73.2017001129822 % Done
73.20708021735621 % Done
73.21246032173025 % Done
73.21784042610426 % Done
73.2232205304783 % Done
73.22860063485231 % Done
73.23398073922634 % Done
73.23936084360037 % Done
73.24474094797439 % Done
73.25012105234842 % Done
73.25550115672243 % Done
73.26088126109647 % Done
73.26626136547048 % Done
73.27164146984451 % Done
73.27702157421854 % Done
73.28240167859256 % Done
73.28778178296659 % Done
73.29316188734062 % Done
73.29854199171464 % Done
73.30392209608867 % Done
73

In [44]:
states=np.vstack([exp[1] for exp in buffer])
states.shape

(2596604, 41)

In [45]:
actions=np.vstack(([exp[2] for exp in buffer]))
rewards=np.vstack(([exp[3] for exp in buffer]))
rewards.shape

(2596604, 1)

In [47]:
df=pd.DataFrame(np.hstack([states,actions,rewards]))

In [48]:
df['pat']=[exp[0] for exp in buffer]

In [50]:
cols=['DBP', 'HR', 'MBP', 'RR', 'SOFA',
 'SpO2', 'SBP','TempC','cardio', 'cns','liver',
 'renal', 'AG','BICARB',	'CREAT',	'CL',	'GLUC',	'HCT',	
           'HB','PLAT'	,'K',	'Na',	'BUN'	,'WBC']+['l_'+str(i) 
           for i in range(1,11)]+['R','C','T','SV', 'age','M/F','weight']


cols=cols+['Vaso','Fluids','Rewards','Pat']

In [54]:
df.to_csv('RL_corrected.csv',index=False)

In [53]:
df.head()

,DBP,HR,MBP,RR,SOFA,SpO2,SBP,TempC,cardio,cns,liver,renal,AG,BICARB,CREAT,CL,GLUC,HCT,HB,PLAT,K,Na,BUN,WBC,l_1,l_2,l_3,l_4,l_5,l_6,l_7,l_8,l_9,l_10,R,C,T,SV,age,M/F,weight,Vaso,Fluids,Rewards,Pat
0,46.0,118.0,62.0,26.0,0.0,94.0,109.0,36.388889,0.0,0.0,0.0,0.0,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462474,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.463221,1.030419,0.419510,65.685165,82.0,0.0,75.1,0.0,0.0,-0.250,262147
1,41.0,111.0,58.0,23.0,2.0,95.0,109.0,36.388889,1.0,0.0,0.0,1.0,1.354087,-0.050696,0.226119,-0.273616,1.664569,0.619855,0.522458,0.450940,2.129651,0.279296,1.282761,-0.258103,-0.462474,-0.086368,0.014108,-0.474435,0.625043,0.389752,0.630310,0.071789,-0.213654,-0.500278,0.461611,0.946364,0.446352,66.568794,82.0,0.0,75.1,0.0,0.0,-0.375,262147
2,44.0,108.0,62.0,19.0,5.0,98.0,118.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266839,0.538169,0.202432,-0.357259,-0.794549,0.509343,0.863491,0.458074,66.941925,82.0,0.0,75.1,0.0,0.0,-0.025,262147
3,41.0,96.0,60.0,20.0,5.0,96.0,118.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266839,0.538169,0.202432,-0.357259,-0.794549,0.554594,0.871751,0.518681,68.092636,82.0,0.0,75.1,0.0,0.0,-0.025,262147
4,46.0,95.0,65.0,20.0,5.0,93.0,124.0,36.388889,1.0,0.0,0.0,4.0,2.643402,-1.035373,0.226119,0.199762,0.582360,0.286389,-0.054658,0.476552,3.521104,0.664734,1.489631,0.661645,-0.696658,0.058330,-0.158973,-0.117952,0.737430,0.266839,0.538169,0.202432,-0.357259,-0.794549,0.609946,0.854549,0.534089,68.271576,82.0,0.0,75.1,0.0,0.0,-0.025,262147


In [27]:
len(cols)

45

In [52]:
df[df['Vaso']>0].shape[0]/df.shape[0]

0.1782651494028354

In [35]:
vaso_cv[vaso_cv.icustay_id.isin(RL_pat_list) & vaso_cv.rate_std==0]

,icustay_id,itemid,charttime,rate_std,starttime
17,200003,30128,2199-08-04 02:00:00,0.000,2199-08-04 02:00:00
18,200003,30128,2199-08-04 02:20:00,0.000,2199-08-04 02:20:00
19,200003,30128,2199-08-04 02:30:00,0.000,2199-08-04 02:30:00
20,200003,30128,2199-08-04 03:00:00,0.000,2199-08-04 03:00:00
21,200003,30128,2199-08-04 03:20:00,0.000,2199-08-04 03:20:00
...,...,...,...,...,...
921906,299999,30128,2117-09-01 11:00:00,0.135,2117-09-01 11:00:00
921907,299999,30128,2117-09-01 11:30:00,0.000,2117-09-01 11:30:00
921908,299999,30128,2117-09-01 12:00:00,0.000,2117-09-01 12:00:00
921909,299999,30128,2117-09-01 13:00:00,0.000,2117-09-01 13:00:00


In [37]:
vaso_MV.vaso_rate.describe()

count    331753.000000
mean          0.306448
std           0.491627
min           0.001000
25%           0.070000
50%           0.200000
75%           0.318000
max          70.695000
Name: vaso_rate, dtype: float64